# Mergesort

In diesem Notebook lernen wir Mergesort kennen und vergleichen das Laufzeitverhalten mit dem eines naiven Sortieralgorithmus, nämlich Selectionsort.

## Merge-Funktion

Der Kern des Mergsort-Algorithmus ist die Mergefunktion, die zwei benachbarte bereits sortierte Bereiche einer Sequenz zusammenführt.

Der erste Bereich geht dabei von Position `lo` bis einschliesslich Position `mid`, der zweite Bereich von Position `mid+1` bis einschliesslich Position `hi`. Array `tmp` dient als "Zwischenspeicher" und muss die gleiche Länge wie `array` haben.

In [ ]:
def merge(array, tmp, lo, mid, hi):
    i = lo
    j = mid + 1
    for k in range(lo, hi + 1):  # k = lo,...,hi
        if j > hi or (i <= mid and array[i] <= array[j]):
            tmp[k] = array[i]
            i += 1
        else:
            tmp[k] = array[j]
            j += 1
    for k in range(lo, hi + 1):  # k = lo,...,hi
        array[k] = tmp[k]

In [ ]:
array = [8, 7, 3, 5, 7, 2, 5, 6, 2, 8]
tmp = [0] * len(array)
merge(array, tmp, 2, 4, 7)
print(array)

## Top-Down-Mergesort

Die Top-Down-Variante von Mergesort teilt den sortierenden Bereich in zwei etwa gleich grosse Teilbereiche auf, sortiert sie jeweils mit einem rekursiven Aufruf und führt die dann sortierten Teilbereiche wieder mit `merge` zusammen.

In [ ]:
def mergesort(array):
    tmp = [0] * len(array)  # [0,...,0] with same size as array
    sort_aux(array, tmp, 0, len(array) - 1)

def sort_aux(array, tmp, lo, hi):
    # print("start sorting positions", lo, "to", hi)
    if hi <= lo:
        return
    mid = lo + (hi - lo) // 2
    # //: Division mit Abrunden
    sort_aux(array, tmp, lo, mid)
    sort_aux(array, tmp, mid + 1, hi)
    merge(array, tmp, lo, mid, hi)
    # print("merged", lo, "-", mid, "and", mid+1, "-", hi)

In [ ]:
array = [4, 2, 5, 7, 9, 6, 4, 1]
mergesort(array)
print(array)

### Visualisierung

Die folgende Visualisierung basiert auf einem Beitrag von Lukas Schaffner, einem früheren Teilnehmer vom GymInf. Damit diese funktioniert, müssen Sie das Graphviz-Paket installieren. Falls Sie die Anaconda-Distribution verwenden, können Sie dies mit folgenden Befehlen machen:
```
conda install -c conda-forge graphviz
conda install -c conda-forge python-graphviz
```

Falls Sie google collab verwenden, installieren Sie das Paket mit folgendem Befehl:
```
!apt-get -qq install -y graphviz && pip install -q pydot
```

In [ ]:
from graphviz import Digraph
from itertools import count

node_counter = count() # used to track the number of nodes in the tree

def sort_draw_tree(array):
    graph = Digraph()
    node_number = next(node_counter)
    graph.node(str(node_number), "sort\n%s" % array)
   
    tmp = [0] * len(array) # [0,...,0] with same size as array
    sort_aux_draw_tree(array, tmp, 0, len(array) - 1, graph, node_number)
    return graph

def sort_aux_draw_tree(array, tmp, lo, hi, graph, parent_node):
    current_node = next(node_counter)
    unsorted_list = str(array[lo:hi+1])
   
    if hi <= lo:
        graph.node(str(current_node), "lo=%s hi=%s \n%s" % (lo, hi, array[lo:hi+1]))
        graph.edge(str(parent_node),str(current_node), str(current_node))
        return
    
    mid = lo + (hi - lo) // 2
    sort_aux_draw_tree(array, tmp, lo, mid, graph, current_node)
    sort_aux_draw_tree(array, tmp, mid + 1, hi, graph, current_node)
    merge(array, tmp, lo, mid, hi)
   
    graph.node(str(current_node), "lo=%s hi=%s \n%s\n%s" % (lo, hi, unsorted_list, array[lo:hi+1]))
    graph.edge(str(parent_node),str(current_node),str(current_node))

In [ ]:
sort_draw_tree([4, 2, 5, 9, 4, 1])

## Bottom-Up-Mergesort

Die Bottom-Up-Variante von Mergesort arbeitet iterativ und sortiert erst alle hintereinanderliegenden Teilbereiche der Grösse 2, dann der Grösse 4, dann 8, etc.

In [ ]:
def bottom_up_mergesort(array):
    n = len(array)
    tmp = [0] * n
    length = 1
    while length < n:
        lo = 0
        while lo < n - length:
            mid = lo + length - 1
            hi = min(lo + 2 * length - 1, n - 1)
            merge(array, tmp, lo, mid, hi)
            # print("merged", lo, "-", mid, "and", mid+1, "-", hi)
            lo += 2 * length
        length *= 2

In [ ]:
array = [4, 2, 5, 7, 9, 6, 4, 1, 5]
bottom_up_mergesort(array)
print(array)

## Laufzeitvergleich Selectionsort vs. Mergesort

Wir wollen nun das Laufzeitverhalten von Selectionsort und Mergesort vergleichen. Hier ist hierfür nochmal die bereits bekannte Implementierung von Selectionsort:

In [ ]:
def selection_sort(array):
    n = len(array)
    for i in range(n - 1):
        min_index = i
        for j in range(i + 1, n):
            if array[j] < array[min_index]:
                min_index = j
        array[i], array[min_index] = array[min_index], array[i]

Im Folgenden messen wir die Laufzeit von Selectionsort für zufällige Arrays der Grösse 10, 100, 1000 und 10000. 

In [ ]:
import timeit
import random

In [ ]:
def create_random_array(n):
    a = list(range(n))
    random.shuffle(a)
    return a

In [ ]:
for i in range (1, 5):
    a = create_random_array(10**i)
    t = timeit.timeit(lambda: selection_sort(a), number=1)
    print("Selectionsort auf %i Elementen benötigt Zeit %s." % (10**i, t))

Wir sehen hier die typische quadratische Laufzeit. Wenn wir 10 mal mehr Elemente sortieren brauchen wir ca. 100 mal länger. 

Wiederholen wir nur das gleiche Experiment mit Mergesort:

In [ ]:
for i in range (1, 7):
    a = create_random_array(10**i)
    t = timeit.timeit(lambda: mergesort(a), number=1)
    print("Mergesort auf %i Elementen benötigt Zeit %s." % (10**i, t))

Im Gegensatz zu Selectionsort sehen wir nun ein sehr viel besseres Laufzeitverhalten. Die Laufzeit steigt nur leicht überlinear. Im Gegensatz zu Selectionsort können wir somit auch sehr grosse Sequenzen effizient sortieren. 